# Lab) Kaggle Competition for Beginner

#### Data Preprocess
- Data Load
- Data Set of Train, Valid, Test
- Feature Engineerning (To Do)
- Data Scaling & Encoding

#### Model Tranding
- Hyper Parameter Tune: Optuna
- Model Traning

#### Ensemble
- Out-of-fold(OOF) Ensemble

#### Submission
- Make submission

In [1]:
# 나눔 글꼴 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 6 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (19.5 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120882 files and direct

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
from os.path import join
import multiprocessing

import numpy as np
import pandas as pd

# 시각화
import matplotlib.pylab as plt
import matplotlib as mpl
import seaborn as sns
import re

%matplotlib inline
#plt.rc('font', family='NanumBarunGothic')

import warnings

warnings.filterwarnings('ignore')

n_cpus = multiprocessing.cpu_count() - 1

##Data Preprocess
### 1. Data Load
- pandas.read_csv()

In [6]:
ROOT_PATH = '/content/drive/MyDrive/ASAC/kaggle특강/실습/'

In [7]:
train_path = join(ROOT_PATH, 'data', 'train.csv')
test_path  = join(ROOT_PATH, 'data', 'test.csv')
submission_path = join(ROOT_PATH, 'data', 'sample_submission.csv')

In [8]:
data = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [9]:
data.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [10]:
test.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,74051,I,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552
1,74052,I,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930
2,74053,F,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415
3,74054,F,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676
4,74055,I,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066


데이터 설명
- id: 각 게 데이터의 고유한 식별자로, 데이터를 구분하는데 사용됩니다.
- Sex: 게의 성별을 나타내는 컬럼으로, 주로 두 가지 값으로 표현됩니다. 예를 들어, "M"은 수컷(male)을, "F"는 암컷(female)을 나타낼 수 있습니다.., "I"는 불확실한 개체
- Length: 게의 길이를 나타내는 컬럼으로, 일반적으로 밀리미터(mm) 단위로 기록됩니다.
- Diameter: 게의 지름을 나타내는 컬럼으로, 마찬가지로 밀리미터(mm) 단위로 기록됩니다.
- Height: 게의 높이를 나타내는 컬럼으로, 역시 밀리미터(mm) 단위로 기록됩니다.
- Weight: 게의 총 무게를 나타내는 컬럼으로, 일반적으로 그램(g) 단위로 표시됩니다.
- Shucked Weight: 게의 껍질을 제거한 후의 무게를 나타내는 컬럼으로, 그램(g) 단위로 기록됩니다.
- Viscera Weight: 게 내장의 무게를 나타내는 컬럼으로, 그램(g) 단위로 표현됩니다.
- Shell Weight: 게의 껍질의 무게를 나타내는 컬럼으로, 그램(g) 단위로 기록됩니다.
- Age(Target): 게의 나이를 나타내는 컬럼으로, 주로 "년"이나 "년수"로 표시되며, 게의 연령을 추정하기 위해 사용됩니다.

In [11]:
# 라벨 컬럼을 별도의 변수로 빼둡니다.
label = data['Age']

In [12]:
data.head(20)

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8
5,5,M,1.5000,1.1750,0.4125,28.845616,13.409313,6.789705,7.937860,10
6,6,M,1.5750,1.1375,0.3500,30.022120,11.935140,7.342521,8.646598,11
7,7,I,1.3125,1.0250,0.3500,18.299602,8.249704,3.898056,5.669900,11
8,8,F,1.6000,1.2875,0.4375,38.824640,16.967176,7.413394,10.772810,12
9,9,M,1.0250,0.7625,0.2625,10.305043,4.493396,2.126212,2.976698,11


In [13]:
data.describe()#include='all')

,id,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
count,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000
mean,37025.000000,1.317460,1.024496,0.348089,23.385217,10.104270,5.058386,6.723870,9.967806
std,21376.826729,0.287757,0.237396,0.092034,12.648153,5.618025,2.792729,3.584372,3.175189
min,0.000000,0.187500,0.137500,0.000000,0.056699,0.028349,0.042524,0.042524,1.000000
25%,18512.500000,1.150000,0.887500,0.300000,13.437663,5.712424,2.863300,3.968930,8.000000
50%,37025.000000,1.375000,1.075000,0.362500,23.799405,9.908150,4.989512,6.931453,10.000000
75%,55537.500000,1.537500,1.200000,0.412500,32.162508,14.033003,6.988152,9.071840,11.000000
max,74050.000000,2.012815,1.612500,2.825000,80.101512,42.184056,21.545620,28.491248,29.000000


In [14]:
#
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74051 entries, 0 to 74050
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              74051 non-null  int64  
 1   Sex             74051 non-null  object 
 2   Length          74051 non-null  float64
 3   Diameter        74051 non-null  float64
 4   Height          74051 non-null  float64
 5   Weight          74051 non-null  float64
 6   Shucked Weight  74051 non-null  float64
 7   Viscera Weight  74051 non-null  float64
 8   Shell Weight    74051 non-null  float64
 9   Age             74051 non-null  int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 5.6+ MB


In [15]:
test.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,74051,I,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552
1,74052,I,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930
2,74053,F,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415
3,74054,F,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676
4,74055,I,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066


In [16]:
test.describe()

,id,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
count,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000
mean,98734.500000,1.319795,1.026357,0.348693,23.464627,10.130220,5.072615,6.745876
std,14251.458382,0.286950,0.236593,0.092259,12.641703,5.611779,2.791563,3.575650
min,74051.000000,0.187500,0.137500,0.000000,0.056699,0.028349,0.014175,0.042524
25%,86392.750000,1.162500,0.887500,0.300000,13.536886,5.740774,2.877474,3.968930
50%,98734.500000,1.387500,1.075000,0.362500,23.813580,9.979024,4.989512,6.945627
75%,111076.250000,1.537500,1.200000,0.412500,32.290081,14.047177,7.016501,9.171063
max,123418.000000,2.037500,1.625000,2.825000,80.101512,42.184056,21.545620,28.491248


In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49368 entries, 0 to 49367
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              49368 non-null  int64  
 1   Sex             49368 non-null  object 
 2   Length          49368 non-null  float64
 3   Diameter        49368 non-null  float64
 4   Height          49368 non-null  float64
 5   Weight          49368 non-null  float64
 6   Shucked Weight  49368 non-null  float64
 7   Viscera Weight  49368 non-null  float64
 8   Shell Weight    49368 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 3.4+ MB


### Feature Engineering(To Do)

In [18]:
# Feature Engineering
# To do

#### 전처리

##### 불필요한 컬럼 삭제
- id 컬럼은 식별자의 역할을 할 뿐 유의미한 feature로 만들기 어렵습니다. 제거하도록 합니다.

In [19]:
data.drop(columns=['id', 'Age'], inplace=True)

test.drop(columns=['id'], inplace=True)

##### 수치형 변수 컬럼 및 범주형 변수 컬럼

In [20]:
# DF.info에서 Object type을 가진 컬럼은 모두 범주형 변수
cat_columns = data.select_dtypes(include='object').columns
num_columns = data.select_dtypes(exclude='object').columns

print('Categorical Columns: \n{}\n\n Numeric Columns: \n{}\n'.format(cat_columns, num_columns))

Categorical Columns: 
Index(['Sex'], dtype='object')

 Numeric Columns: 
Index(['Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight'],
      dtype='object')



In [21]:
data.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928
1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940
2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133
3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885
4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395


In [22]:
test.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,I,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552
1,I,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930
2,F,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415
3,F,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676
4,I,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066


# 베이스라인 구축 (Pycaret)

In [23]:
!pip install -q pycaret
!pip install --upgrade -q xgboost
!pip install -q catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.7/484.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 106.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.6 MB/

In [26]:
!pip install scipy

In [27]:
from pycaret.regression import *

X = pd.concat([data, label], axis = 1)

In [28]:
X.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [29]:
from matplotlib.colors import Normalize
reg = setup(data = X # 데이터
      ,target = 'Age' # Target
      ,verbose = True # 진행 로그 출력
      ,train_size = 0.7 # 내부적으로 검증시 훈련데이터의 비율
      ,data_split_shuffle = True # 데이터 split시 데이터를 섞을것인지 여부
      ,normalize = True # 정규화 여부
      ,session_id = 100 # 난수 시드
      ,use_gpu= True )

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

,Description,Value
0,Session id,100
1,Target,Age
2,Target type,Regression
3,Original data shape,"(74051, 9)"
4,Transformed data shape,"(74051, 11)"
5,Transformed train set shape,"(51835, 11)"
6,Transformed test set shape,"(22216, 11)"
7,Numeric features,7
8,Categorical features,1
9,Preprocess,True


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

In [30]:
models()

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [31]:
top_5_model = compare_models(fold = 5, round = 3, sort = 'mse', n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,1.407,4.212,2.052,0.582,0.163,0.137,0.436
gbr,Gradient Boosting Regressor,1.423,4.260,2.064,0.577,0.164,0.139,5.158
catboost,CatBoost Regressor,1.422,4.265,2.065,0.577,0.164,0.139,9.524
xgboost,Extreme Gradient Boosting,1.431,4.361,2.088,0.567,0.165,0.139,0.728
rf,Random Forest Regressor,1.471,4.486,2.118,0.555,0.170,0.145,4.542
lr,Linear Regression,1.482,4.505,2.122,0.553,0.169,0.145,0.186
ridge,Ridge Regression,1.482,4.505,2.122,0.553,0.169,0.145,0.172
lar,Least Angle Regression,1.482,4.505,2.122,0.553,0.169,0.145,0.164
br,Bayesian Ridge,1.482,4.505,2.122,0.553,0.169,0.145,0.242
et,Extra Trees Regressor,1.482,4.525,2.127,0.551,0.171,0.146,2.112


Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [32]:
model_rf = create_model('rf', fold = 10)
model_cb = create_model('catboost', fold = 10)
model_lgb = create_model('lightgbm', fold = 10)
model_xgb = create_model('xgboost', fold = 10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.4792,4.4932,2.1197,0.5575,0.1684,0.1447
1,1.4587,4.3534,2.0865,0.5609,0.1684,0.1448
2,1.4946,4.6000,2.1448,0.5514,0.1730,0.1477
3,1.5058,4.7313,2.1752,0.5399,0.1720,0.1456
4,1.4733,4.5261,2.1275,0.5555,0.1711,0.1463
5,1.4343,4.1453,2.0360,0.5747,0.1655,0.1438
6,1.4541,4.4827,2.1172,0.5485,0.1681,0.1417
7,1.4801,4.5729,2.1384,0.5638,0.1690,0.1440
8,1.4737,4.4978,2.1208,0.5470,0.1701,0.1466


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.4265,4.2867,2.0704,0.5778,0.1632,0.1380
1,1.4124,4.1667,2.0413,0.5797,0.1632,0.1390
2,1.4494,4.3463,2.0848,0.5761,0.1668,0.1421
3,1.4595,4.5501,2.1331,0.5575,0.1663,0.1395
4,1.4140,4.2427,2.0598,0.5834,0.1643,0.1394
5,1.3652,3.8082,1.9515,0.6093,0.1571,0.1352
6,1.4080,4.2640,2.0649,0.5705,0.1624,0.1357
7,1.4417,4.4548,2.1106,0.5751,0.1648,0.1380
8,1.4272,4.2809,2.0690,0.5689,0.1650,0.1406


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.4109,4.2306,2.0568,0.5834,0.1620,0.1364
1,1.3907,4.0827,2.0206,0.5882,0.1614,0.1367
2,1.4270,4.2600,2.0640,0.5845,0.1652,0.1398
3,1.4471,4.5223,2.1266,0.5602,0.1656,0.1381
4,1.3987,4.1697,2.0420,0.5905,0.1630,0.1378
5,1.3520,3.7530,1.9373,0.6150,0.1562,0.1342
6,1.3941,4.2439,2.0601,0.5725,0.1618,0.1342
7,1.4213,4.3477,2.0851,0.5853,0.1632,0.1363
8,1.4107,4.2250,2.0555,0.5745,0.1635,0.1388


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.4448,4.4462,2.1086,0.5621,0.1654,0.1393
1,1.4105,4.2068,2.0511,0.5757,0.1639,0.1388
2,1.4437,4.4119,2.1004,0.5697,0.1674,0.1410
3,1.4614,4.6182,2.1490,0.5509,0.1670,0.1390
4,1.4272,4.3479,2.0852,0.5730,0.1661,0.1404
5,1.3872,3.9758,1.9939,0.5921,0.1601,0.1374
6,1.4141,4.3717,2.0909,0.5596,0.1639,0.1359
7,1.4359,4.4500,2.1095,0.5756,0.1650,0.1376
8,1.4185,4.2961,2.0727,0.5673,0.1646,0.1395


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# tune_model_cb = tune_model(model_cb, n_iter = 50, fold = 5, optimize = 'mse', choose_better = True)
# tune_model_lgb = tune_model(model_lgb, n_iter = 50, fold = 5, optimize = 'mse', choose_better = True)
# tune_model_xgb = tune_model(model_xgb, n_iter = 50, fold = 5, optimize = 'mse', choose_better = True)

In [34]:
candidate_models = [model_cb, model_lgb, model_xgb, model_rf]

In [35]:
stacker = stack_models(candidate_models, fold = 10, choose_better = True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.4130,4.2202,2.0543,0.5844,0.1619,0.1367
1,1.3888,4.0579,2.0144,0.5907,0.1612,0.1367
2,1.4274,4.2318,2.0571,0.5873,0.1649,0.1400
3,1.4424,4.4695,2.1141,0.5654,0.1648,0.1378
4,1.3959,4.1499,2.0371,0.5925,0.1627,0.1376
5,1.3501,3.7425,1.9346,0.6161,0.1560,0.1341
6,1.3914,4.2040,2.0504,0.5765,0.1610,0.1339
7,1.4202,4.3301,2.0809,0.5870,0.1628,0.1362
8,1.4073,4.1798,2.0445,0.5791,0.1630,0.1387


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [36]:
stacker_false = stack_models(candidate_models, fold = 10, choose_better = True, restack = False)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.4113,4.2196,2.0542,0.5844,0.1618,0.1364
1,1.3899,4.0662,2.0165,0.5898,0.1612,0.1367
2,1.4282,4.2533,2.0624,0.5852,0.1651,0.1399
3,1.4419,4.4827,2.1172,0.5641,0.1649,0.1377
4,1.3980,4.1692,2.0419,0.5906,0.1629,0.1377
5,1.3508,3.7444,1.9350,0.6159,0.1560,0.1340
6,1.3897,4.2037,2.0503,0.5766,0.1611,0.1338
7,1.4186,4.3272,2.0802,0.5873,0.1627,0.1360
8,1.4088,4.1966,2.0486,0.5774,0.1631,0.1387


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [37]:
stack_model = finalize_model(stacker)
stack_model

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Length', 'Diameter', 'Height',
                                             'Weight', 'Shucked Weight',
                                             'Viscera Weight', 'Shell Weight'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['Sex'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('onehot_encoding',
                 Transforme...
                                                             max_delta_step=None,
                                                             max_depth=None,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=None,
                                                             n_jobs=-1,
                                                             num_parallel_tree=None,
                                                             random_state=100, ...)),
                                               ('Random Forest Regressor',
                                                RandomForestRegressor(n_jobs=-1,
                                                                      random_state=100))],
                                   final_estimator=LinearRegression(n_jobs=-1),
                                   n_jobs=1, passthrough=True))])

In [38]:
test

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,I,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552
1,I,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930
2,F,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415
3,F,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676
4,I,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066
...,...,...,...,...,...,...,...,...
49363,F,1.3000,1.0375,0.3250,16.315137,6.690482,5.173784,3.756309
49364,I,1.0375,0.7625,0.2625,10.276694,4.436697,1.998640,3.543687
49365,F,1.4875,1.1625,0.3625,31.382897,11.396499,6.846404,8.788345
49366,F,1.2375,0.9500,0.2875,15.663099,6.095142,3.727959,4.961163


In [39]:
pred_test = predict_model(stack_model, data = test)
pred_test

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,prediction_label
0,I,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552,7.617479
1,I,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930,7.797796
2,F,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415,10.813618
3,F,1.5500,0.9875,0.3875,28.377850,13.380964,6.548735,7.030676,9.687049
4,I,1.1125,0.8500,0.2625,11.765042,5.528152,2.466407,3.331066,7.504417
...,...,...,...,...,...,...,...,...,...
49363,F,1.3000,1.0375,0.3250,16.315138,6.690482,5.173784,3.756309,9.493702
49364,I,1.0375,0.7625,0.2625,10.276693,4.436697,1.998640,3.543688,8.060650
49365,F,1.4875,1.1625,0.3625,31.382896,11.396499,6.846404,8.788345,13.239380
49366,F,1.2375,0.9500,0.2875,15.663098,6.095142,3.727959,4.961163,9.947388


In [57]:
prediction_round = round(pred_test.prediction_label)

In [46]:
label.unique() # 기존 라벨은 모두 정수이다.

array([ 9,  8, 11, 10, 12,  7,  5, 14,  6, 13, 16,  4, 18, 15, 17,  3, 19,
       23, 21, 20, 26, 22, 27, 24, 25, 29,  2,  1])

### Submission

In [49]:
submit = pd.read_csv(join(ROOT_PATH, 'data','sample_submission.csv'))

In [50]:
submit['Age'] = pred_test.prediction_label

In [52]:
submit.to_csv(join(ROOT_PATH, 'submissions','submission_1.csv'), index=False) # 1.4031

In [58]:
submit['Age'] = prediction_round

In [59]:
submit.to_csv(join(ROOT_PATH, 'submissions','submission_2.csv'), index=False) # 1.38366